# EDA & Model Development 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import random

## Initial Setup

In [2]:
#https://raw.githubusercontent.com/jchristo12/fantasy_football/master/data/full_data.csv
df = pd.read_csv('https://github.com/jchristo12/fantasy_football/blob/master/data/full_data.csv?raw=true')

In [3]:
df.columns

Index(['pk', 'gid', 'seas', 'wk', 'player', 'fname', 'lname', 'full_name',
       'team', 'pos1',
       ...
       'humd', 'ou', 'sprv', 'ptsv', 'ptsh', 'udog', 'gen_cond', 'udog_binary',
       'def_team', 'f_pts'],
      dtype='object', length=161)

In [4]:
#remove rows that have NaN for the shifted variables
df_clean = df[~df.loc[:,'seas_pa':'seas_tdret'].isna().all(axis=1)]

##### We are only focused on offensive players right now. Therefore, we will specifiy the positions we want to retain in the data and remove everyone else

In [5]:
#store positions we are concerned about; will use these to filter out 
pos_of_interest = ['QB', 'RB', 'WR', 'TE']
#filter out positions we don't care about
df_clean2 = df_clean[df_clean['pos1'].isin(pos_of_interest)]

##### Convert each column to the appropriate data type (i.e. make the categorical data categories)

In [6]:
#set the column types
col_dtypes = {'category': ['seas', 'wk', 'pos1', 'team', 'udog', 'dv', 'v', 'h', 'day', 'cond', 'stad', 'wdir',
                          'surf', 'gen_cond', 'def_team']}
#flip the key and values around so they will work in the argument for 'astype()'
col_dtypes_alt = {old: new for new, old_all in col_dtypes.items() for old in old_all}

In [7]:
df_clean2 = df_clean2.astype(col_dtypes_alt)

##### Segment out the WR data

In [8]:
df_wr = df_clean2.loc[df_clean2['pos1']=='WR', :]

##### Only focus on week 10 data

In [9]:
df_wr10 = df_wr.loc[df_wr['wk'] == 10, :]

## EDA

Only focusing on the WR data right now. Will need to build this so that each action is extendable to the other positions

### Training/Test split

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
#set the random seed for reproducability
random.seed(837)

In [12]:
#break out the data between training and test
train_wr, test_wr = train_test_split(df_wr10, train_size=0.75, test_size=0.25, shuffle=True)

In [13]:
#shape of the data
print(train_wr.shape, test_wr.shape)

(1625, 161) (542, 161)


### Missing Data analysis

#### Basic analysis

In [14]:
#create a series of percent of missing data
missing_data_pct = train_wr.isna().sum() / train_wr.shape[0]

In [15]:
#list all columns with missing data that is greater than 25%
missing_data_pct[missing_data_pct > 0.25].sort_values(ascending=False)

roll_td_to_int           0.990154
seas_td_to_int           0.988308
roll_yds_per_comp        0.986462
seas_yds_per_comp        0.980923
roll_comp_pct            0.972308
seas_comp_pct            0.960000
career_td_to_int         0.849231
career_yds_per_comp      0.835692
career_comp_pct          0.791385
roll_carry_to_td         0.734769
roll_ryds_per_carry      0.734154
roll_ret_to_td           0.681231
roll_avg_ret             0.680615
roll_carry_to_fuml       0.664615
seas_ret_to_td           0.648000
seas_avg_ret             0.647385
seas_carry_to_td         0.633231
seas_ryds_per_carry      0.632615
career_avg_ret           0.592000
career_ret_to_td         0.592000
seas_carry_to_fuml       0.539692
career_ryds_per_carry    0.312615
career_carry_to_td       0.311385
dtype: float64

In [16]:
#columns with missing values but less than or equal to 25%
missing_data_pct[(missing_data_pct <= 0.25) & (missing_data_pct > 0)].sort_values(ascending=False)

humd                    0.238154
career_carry_to_fuml    0.235077
wdir                    0.227077
career_rec_to_td        0.145846
temp                    0.143385
career_yds_per_rec      0.140923
career_catch_pct        0.131077
last_ints               0.129231
last_fuml               0.129231
last_rety               0.129231
last_ret                0.129231
last_tdrec              0.129231
last_recy               0.129231
last_rec                0.129231
last_trg                0.129231
last_tdr                0.129231
last_tdp                0.129231
last_ry                 0.129231
last_pa                 0.129231
last_pc                 0.129231
last_sra                0.129231
last_ra                 0.129231
last_py                 0.129231
last_tdret              0.129231
dv                      0.076923
roll_rec_to_td          0.070154
roll_yds_per_rec        0.070154
seas_rec_to_td          0.063385
seas_yds_per_rec        0.063385
roll_catch_pct          0.044923
seas_catch

In [55]:
train_wr_miss.loc[train_wr_miss['last_rec'].isna(), :]

,pk,gid,seas,wk,player,fname,lname,full_name,team,pos1,...,humd,ou,sprv,ptsv,ptsh,udog,gen_cond,udog_binary,def_team,f_pts
40950,4661&JD-2350,4661,2017,10,JD-2350,Josh,Doctson,Josh Doctson,WAS,WR,...,44.0,41.0,-1.5,38,30,WAS,fair_cond,True,MIN,3.000000
54605,3333&KM-0450,3333,2012,10,KM-0450,Keshawn,Martin,Keshawn Martin,HOU,WR,...,90.0,39.5,1.0,13,6,HOU,percip_cond,True,CHI,2.857143
74795,1993&RC-3100,1993,2007,10,RC-3100,Ronald,Curry,Ronald Curry,OAK,WR,...,63.0,38.0,-3.0,17,6,OAK,fair_cond,True,CHI,2.000000
241,2797&AB-0800,2797,2010,10,AB-0800,Arnaz,Battle,Arnaz Battle,PIT,WR,...,86.0,45.0,4.5,39,26,NE,fair_cond,False,NE,0.000000
34584,4392&FM-0050,4392,2016,10,FM-0050,Freddie,Martino,Freddie Martino,TB,WR,...,61.0,45.5,-2.5,10,36,TB,fair_cond,True,CHI,10.300000
3796,3858&AJ-0460,3858,2014,10,AJ-0460,A.J.,Jenkins,A.J. Jenkins,KC,WR,...,72.0,42.0,-1.0,17,13,BUF,fair_cond,False,BUF,2.700000
5349,4395&AR-1250,4395,2016,10,AR-1250,Allen,Robinson,Allen Robinson,JAC,WR,...,NaN,42.0,2.5,24,21,HOU,percip_cond,False,HOU,18.700000
33217,1988&EM-1800,1988,2007,10,EM-1800,Eric,Moulds,Eric Moulds,TEN,WR,...,30.0,35.0,4.0,28,13,JAC,fair_cond,False,JAC,1.800000
14891,923&CC-4100,923,2003,10,CC-4100,Curtis,Conway,Curtis Conway,NYJ,WR,...,75.0,38.0,-3.0,27,24,OAK,fair_cond,False,OAK,5.400000
27342,2531&DM-0500,2531,2009,10,DM-0500,Derrick,Mason,Derrick Mason,BAL,WR,...,NaN,40.0,-10.0,16,0,CLE,fair_cond,False,CLE,7.800000


#### Take action on missing data

##### Drop columns with too many missing values

In [17]:
#drop columns with missing data greater than 25%
#store the column names
missing_cols_del = missing_data_pct[missing_data_pct > 0.25].sort_values(ascending=False).index
#drop the columns and store as new dataframe
train_wr_miss = train_wr.drop(missing_cols_del, axis=1, inplace=False)

In [18]:
#check the shape
train_wr_miss.shape

(1625, 138)

##### Impute the rest of the missing values

In [86]:
def tree_impute(df, cols):
    #import necessary packages
    try:
        from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
        from sklearn.impute import SimpleImputer
        import pandas as pd
    except:
        print('Necessary packages are not installed. Please install:')
        print('scikit-learn & pandas')
    
    #initialize a list to store imputed features
    imputed_features = []
    
    #set up the dataframe to handle categorical features in Decision Trees
        #store the categorical and bool columns as OneHotEncoded dummies

    
    #loop thru all features that have missing values
    for c in cols:
        #store the column datatype
        data_type = df[c].dtype
        #store the target variable separate
        target = df[c].values
        
        
        #store the features separate
        features_df = df.drop(c, axis=1)
        #create one hot encoded dummy variables
        dummies = pd.get_dummies(features_df.select_dtypes(include=['category', 'bool']), drop_first=True)
        #drop original columns (as well as strings) and concate the dummies
        interim_df = features_df.drop(features_df.select_dtypes(include=['category', 'bool']), axis=1).select_dtypes(exclude='object')
        new_df = pd.concat([interim_df, dummies], axis=1)
        #convert to values
        features = new_df.values
        
        
        
        #perform the imputation
        if (data_type == 'category') | (data_type == 'bool'):
            #Decision tree for categorical variable
            Ctree = DecisionTreeClassifier(criterion='gini', random_state=100, max_depth=5, min_samples_leaf=3)
            Ctree.fit(features, target)
            output = Ctree.predict(target)
        else:
            try:
                #Decision tree for regression variables
                Rtree = DecisionTreeRegressor(criterion='mse', random_state=100, max_depth=5, min_samples_leaf=3)
                Rtree.fit(features, target)
                output = Rtree.predict(target)
            except:
                #use the average value to impute missing values
                print('Decision tree could not be used for imputation')
                print('Average value used')
                mean_impute = SimpleImputer(strategy='mean')
                output = mean_impute.fit_transform(target)
        
        #store the missing values
        result = pd.Series(output).rename('imp_' + c)
        imputed_features.append(result)
    
    #concat all series into a dataframe
    final_df = pd.concat(imputed_features, axis=1)
    
    return final_df

In [19]:
#store the columns with missing data
low_missing_cols = missing_data_pct[(missing_data_pct <= 0.25) & (missing_data_pct > 0)].index

Testing

In [43]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.impute import SimpleImputer
import pandas as pd

In [44]:
low_missing_cols

Index(['dv', 'last_pa', 'last_pc', 'last_py', 'last_ints', 'last_tdp',
       'last_ra', 'last_sra', 'last_ry', 'last_tdr', 'last_fuml', 'last_trg',
       'last_rec', 'last_recy', 'last_tdrec', 'last_ret', 'last_rety',
       'last_tdret', 'career_pa', 'career_pc', 'career_py', 'career_ints',
       'career_tdp', 'career_ra', 'career_sra', 'career_ry', 'career_tdr',
       'career_fuml', 'career_trg', 'career_rec', 'career_recy',
       'career_tdrec', 'career_ret', 'career_rety', 'career_tdret',
       'career_carry_to_fuml', 'career_catch_pct', 'seas_catch_pct',
       'roll_catch_pct', 'career_yds_per_rec', 'seas_yds_per_rec',
       'roll_yds_per_rec', 'career_rec_to_td', 'seas_rec_to_td',
       'roll_rec_to_td', 'cond', 'temp', 'wdir', 'humd'],
      dtype='object')

In [45]:
#store the argments for testing
c = low_missing_cols[0]
df = train_wr_miss

In [46]:
#store the column type
col_data_type = df[c].dtype

In [47]:
#create dummy variables
#dummies = pd.get_dummies(df.select_dtypes(include=['category', 'bool']), drop_first=True)

In [30]:
#drop original columns (as well as strings) and concate the dummies
interim_df = df.drop(df.select_dtypes(include=['category', 'bool']), axis=1).select_dtypes(exclude='object')
new_df = pd.concat([interim_df, dummies], axis=1)
#drop identifier columns
new_df = new_df.drop(['gid', 'nflid'], axis=1)
#drop ultimate response variable
new_df = new_df.drop('f_pts', axis=1)

In [31]:
for i in new_df.columns: print(i)

pa
pc
py
ints
tdp
ra
sra
ry
tdr
fuml
trg
rec
recy
tdrec
ret
rety
tdret
exp
height
weight
forty
bench
vertical
broad
shuttle
cone
arm
hand
seas_pa
seas_pc
seas_py
seas_ints
seas_tdp
seas_ra
seas_sra
seas_ry
seas_tdr
seas_fuml
seas_trg
seas_rec
seas_recy
seas_tdrec
seas_ret
seas_rety
seas_tdret
last_pa
last_pc
last_py
last_ints
last_tdp
last_ra
last_sra
last_ry
last_tdr
last_fuml
last_trg
last_rec
last_recy
last_tdrec
last_ret
last_rety
last_tdret
career_pa
career_pc
career_py
career_ints
career_tdp
career_ra
career_sra
career_ry
career_tdr
career_fuml
career_trg
career_rec
career_recy
career_tdrec
career_ret
career_rety
career_tdret
recent_fuml
recent_ints
recent_pa
recent_pc
recent_py
recent_ra
recent_rec
recent_recy
recent_ret
recent_rety
recent_ry
recent_sra
recent_tdp
recent_tdr
recent_tdrec
recent_tdret
recent_trg
career_carry_to_fuml
career_catch_pct
seas_catch_pct
roll_catch_pct
career_yds_per_rec
seas_yds_per_rec
roll_yds_per_rec
career_rec_to_td
seas_rec_to_td
roll_rec_to_td
ag

In [48]:
#store the target
target = df[c].values

In [49]:
#store the features
features_df = df.drop(c, axis=1)

In [50]:
#create one hot encoded dummy variables
dummies = pd.get_dummies(features_df.select_dtypes(include=['category', 'bool']), drop_first=False)

In [51]:
#drop original columns (as well as strings) and concate the dummies
interim_df = features_df.drop(features_df.select_dtypes(include=['category', 'bool']), axis=1).select_dtypes(exclude='object')
new_df = pd.concat([interim_df, dummies], axis=1)

In [52]:
#convert to values
features = new_df.values

In [53]:
#tree classifier
tree = DecisionTreeClassifier(criterion='gini', random_state=212, min_samples_leaf=3)

In [54]:
#fit the tree
tree.fit(features, target)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').